<a href="https://colab.research.google.com/github/pedroafleite/data_etl/blob/main/data_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install flatten_json

  Created wheel for flatten-json: filename=flatten_json-0.1.13-py3-none-any.whl size=7979 sha256=e717de244d0d0730ead41bc063df76623afd22bfd26bfddb5357e88cd2012d2c
  Stored in directory: /root/.cache/pip/wheels/e6/b3/2a/beb2ceb72d11bf335f9c2f87aae26981f6744f3fc885cde665
Successfully built flatten-json


In [4]:
import pandas as pd
import json
from flatten_json import flatten

In [5]:
infomix = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_etl/data_etl_test/infomix.tsv', sep='\t')
infomix.head()

,cnpj,gtin,category
0,8540603000198,6890101502534,massas e molhos
1,71322150004408,7898605250417,cervejas
2,43235985000732,7896000724465,cabelo
3,47508411134004,7896102589122,conservas e enlatados
4,47508411135833,7895000474646,bomboniere


In [6]:
cnpj_raw = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data_etl/data_etl_test/cnpjs_receita_federal.jl', lines=True)
cnpj_json = json.loads(cnpj_raw.to_json(orient="records"))
cnpj_flat = (flatten(record, '.') for record in cnpj_json)
cnpj = pd.DataFrame(cnpj_flat)
cnpj.head()

# Razao social: response.nome
# CNPJ: cnpj
# Cidade: response.municipio
# Estado: response.uf

,id,cnpj,response.atividades_secundarias.0.code,response.atividades_secundarias.0.text,response.atividades_secundarias.1.code,response.atividades_secundarias.1.text,response.atividades_secundarias.2.code,response.atividades_secundarias.2.text,response.motivo_situacao,response.google_place_id,response.latitude,response.tipo,response.capital_social,response.atividade_principal.0.code,response.atividade_principal.0.text,response.email,response.natureza_juridica,response.numero,response.municipio,response.uf,response.telefone,response.data_situacao,response.logradouro,response.status,response.situacao,response.situacao_especial,response.longitude,response.extra,response.abertura,response.data_situacao_especial,response.nome,response.complemento,response.ultima_atualizacao,response.bairro,response.qsa.0.nome,response.qsa.0.qual,response.qsa.1.nome,response.qsa.1.qual,response.qsa.2.nome,response.qsa.2.qual,...,response.atividades_secundarias.72.text,response.atividades_secundarias.72.code,response.atividades_secundarias.73.text,response.atividades_secundarias.73.code,response.atividades_secundarias.74.text,response.atividades_secundarias.74.code,response.atividades_secundarias.75.text,response.atividades_secundarias.75.code,response.atividades_secundarias.76.text,response.atividades_secundarias.76.code,response.atividades_secundarias.77.text,response.atividades_secundarias.77.code,response.atividades_secundarias.78.text,response.atividades_secundarias.78.code,response.atividades_secundarias.79.text,response.atividades_secundarias.79.code,response.atividades_secundarias.80.text,response.atividades_secundarias.80.code,response.atividades_secundarias.81.text,response.atividades_secundarias.81.code,response.atividades_secundarias.82.text,response.atividades_secundarias.82.code,response.atividades_secundarias.83.text,response.atividades_secundarias.83.code,response.atividades_secundarias.84.text,response.atividades_secundarias.84.code,response.qsa.10.qual_rep_legal,response.qsa.10.nome_rep_legal,response.qsa.11.qual_rep_legal,response.qsa.11.nome_rep_legal,response.qsa.6.pais_origem,response.message,response.atividades_secundarias.85.text,response.atividades_secundarias.85.code,response.atividades_secundarias.86.text,response.atividades_secundarias.86.code,response.atividades_secundarias.87.text,response.atividades_secundarias.87.code,response.qsa.12.nome_rep_legal,response.qsa.12.qual_rep_legal
0,99068e55-1018-47e7-91a9-44cded045033,68093095000179,10.53-8-00,Fabricação de sorvetes e outros gelados comest...,52.11-7-99,"Depósitos de mercadorias para terceiros, excet...",47.29-6-99,Comércio varejista de produtos alimentícios em...,,ChIJby61NifBzpQRVvO7DKFIYTM,-23.117549,MATRIZ,39950000.00,10.31-7-00,Fabricação de conservas de frutas,fiscal@frooty.com.br,205-4 - Sociedade Anônima Fechada,S/N,ATIBAIA,SP,(11) 3093-3100 / (11) 3093-1300,03/11/2005,ESTM CAMPO LIMPO PAULISTA A ATIBAIA,OK,ATIVA,,-46.556707,{},01/07/1992,,FROOTY COMERCIO E INDUSTRIA DE ALIMENTOS S.A.,,2018-08-09T18:58:13.133Z,IARA,MARCELO CESANA,10-Diretor,SILVIO LA ROCCA SANTOS,10-Diretor,PEDRO HENRIQUE CHRISTOVAM GOMES,10-Diretor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c475b934-eef1-488e-8f6a-b6087ef32b9b,13492669000190,46.39-7-01,Comércio atacadista de produtos alimentícios e...,64.62-0-00,Holdings de instituições não-financeiras,46.49-4-99,Comércio atacadista de outros equipamentos e a...,,ChIJ1bUkuoVXzpQR9SpW9Y0qwJI,-23.554987,MATRIZ,76729078.00,46.35-4-03,Comércio atacadista de bebidas com atividade d...,financeiro@estrellagalicia.com.br,206-2 - Sociedade Empresária Limitada,359,SAO PAULO,SP,(11) 2368-5876,17/03/2011,R CARDEAL ARCOVERDE,OK,ATIVA,,-46.676754,{},17/03/2011,,ESTRELLA DE GALICIA IMPORTACAO E COMERCIALIZAC...,ANDAR 6 CONJ 62 63 64 ANDAR 7 CONJ 71 72 73 74,2018-09-05T15:13:52.878Z,PINHEIROS,HIJOS DE RIVERA SA,37-Sócio Pessoa Jurídica Domi

In [7]:
gs1_raw = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data_etl/data_etl_test/gs1.jl', lines=True)
gs1_json = json.loads(gs1_raw.to_json(orient="records"))
gs1_flat = (flatten(record, '.') for record in gs1_json)
gs1 = pd.DataFrame(gs1_flat)
gs1.head()

,id,gtin,response.status,response.gepirParty.partyDataLine.address.city,response.gepirParty.partyDataLine.address.name,response.gepirParty.partyDataLine.address.state,response.gepirParty.partyDataLine.address.cityCode,response.gepirParty.partyDataLine.address.countyCode,response.gepirParty.partyDataLine.address.postalCode,response.gepirParty.partyDataLine.address.countryCode._,response.gepirParty.partyDataLine.address.countryCode.codeListVersion,response.gepirParty.partyDataLine.address.crossStreet,response.gepirParty.partyDataLine.address.pOBoxNumber,response.gepirParty.partyDataLine.address.provinceCode,response.gepirParty.partyDataLine.address.streetAddressOne,response.gepirParty.partyDataLine.address.streetAddressTwo,response.gepirParty.partyDataLine.address.streetAddressThree,response.gepirParty.partyDataLine.address.currencyOfPartyCode._,response.gepirParty.partyDataLine.address.currencyOfPartyCode.codeListVersion,response.gepirParty.partyDataLine.address.languageOfThePartyCode._,response.gepirParty.partyDataLine.address.languageOfThePartyCode.codeListVersion,response.gepirParty.partyDataLine.address.geographicalCoordinates.latitude,response.gepirParty.partyDataLine.address.geographicalCoordinates.longitude,response.gepirParty.partyDataLine.returnCode._,response.gepirParty.partyDataLine.returnCode.codeListVersion,response.gepirParty.partyDataLine.gS1KeyLicensee.gln,response.gepirParty.partyDataLine.gS1KeyLicensee.partyName,response.gepirParty.partyDataLine.gS1KeyLicensee.partyRole._,response.gepirParty.partyDataLine.gS1KeyLicensee.partyRole.codeListVersion,response.gepirParty.partyDataLine.gS1KeyLicensee.additionalPartyIdentification._,response.gepirParty.partyDataLine.gepirRequestedKey.requestedKeyCode._,response.gepirParty.partyDataLine.gepirRequestedKey.requestedKeyValue,response.gepirParty.partyDataLine.partyDataLanguage._,response.gepirParty.partyDataLine.partyDataLanguage.codeListVersion,response.gepirParty.partyDataLine.countryAdministered,response.gepirParty.partyDataLine.informationProvider.gln,response.gepirParty.partyDataLine.informationProvider.partyName,response.gepirParty.partyDataLine.responderSpecificData.responderSpecificData,response.gepirParty.partyDataLine.responderSpecificData.responderSpecificDataUse,response.gepirParty.partyDataLine.gS1CompanyPrefixLicensee.gln,...,response.gepirParty.partyDataLine.contact.communicationChannel.communicationValue,response.gepirParty.partyDataLine.contact.communicationChannel.communicationChannelCode._,response.gepirParty.partyDataLine.contact.communicationChannel.communicationChannelCode.codeListVersion,response.gepirParty.partyDataLine.contact.communicationChannel.communicationChannelName,response.gepirParty.partyDataLine.responderSpecificData,response.gepirParty.partyDataLine.gepirItemExternalFileLink.0.fileFormatName._,response.gepirParty.partyDataLine.gepirItemExternalFileLink.0.uniformResourceIdentifier,response.gepirParty.partyDataLine.gepirItemExternalFileLink.1.fileFormatName._,response.gepirParty.partyDataLine.gepirItemExternalFileLink.1.uniformResourceIdentifier,response.gepirParty.partyDataLine.contact.jobTitle,response.gepirParty.partyDataLine.gepirRequestedKey.requestedKeyCode.codeListVersion,response.gepirParty.partyDataLine.gepirRequestedKey.requestedLanguage._,response.gepirParty.partyDataLine.gepirRequestedKey.requestedLanguage.codeListVersion,response.error.code,response.error.message,response.error.extendedHelp,response.gepirParty.partyDataLine.contact.departmentName,response.gepirParty.partyDataLine.contact.responsibility.codeListVersion,response.gepirParty.partyDataLine.gS1KeyLicensee.partyRole.0._,response.gepirParty.partyDataLine.gS1KeyLicensee.partyRole.0.codeListVersion,response.gepirParty.partyDataLine.gS1KeyLicensee.partyRole.1._,response.gepirParty.partyDataLine.gS1KeyLicensee.partyRole.1.codeListVersion,response.gepirParty.partyDataLine.gS1CompanyPrefixLicensee.partyRole.0._,response.gepirParty.partyDataLine.gS1CompanyPrefixLicensee.part

In [11]:
descricoes = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_etl/data_etl_test/descricoes_externas.tsv', sep='\t')
descricoes.head()

,id,gtin,description,flag_infoprice
0,2482686,7896512911933,SABONETE INFANTIL LIQUIDO GRANADO BEBE LAVANDA...,True
1,2482685,7896512911933,SAB GRANADO BEBE LAVANDA 250G,False
2,2482684,7896512911933,SAB LIQ.GRAN.BEBE LAVAN 25,False
3,2482683,7896512911933,SAB GRANADO GLICLAVAN BEBE 250M,False
4,2482682,7896512911933,SAB BABY GRA LA 250ML,False


In [12]:
cosmos_raw = pd.read_json('/content/drive/MyDrive/Colab Notebooks/data_etl/data_etl_test/cosmos.jl', lines=True)
cosmos_json = json.loads(cosmos_raw.to_json(orient="records"))
cosmos_flat = (flatten(record, '.') for record in cosmos_json)
cosmos = pd.DataFrame(cosmos_flat)
cosmos.head()

,id,gtin,response.gcp.code,response.gcp.description,response.ncm.code,response.ncm.description,response.gtin,response.brand,response.image,response.units.0.gtin,response.units.0.type,response.units.0.packing_size,response.status,response.autited,response.description,created_at,response.units.1.gtin,response.units.1.type,response.units.1.packing_size,response.units.2.gtin,response.units.2.type,response.units.2.packing_size,response.units.3.gtin,response.units.3.type,response.units.3.packing_size,response.units
0,16bde255-988c-466b-9d35-39b706b9fbda,7896040700276,10000424,Detergentes para a roupa,3402.20.00,Preparações acondicionadas para venda a retalho,07896040700276,None,https://cdn-cosmos.bluesoft.com.br/products/78...,7896040700276,Unidade,1,OK,,LAVA ROUPAS COQUETEL ERVAS,1540236268000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,96f61c2b-5057-439c-9717-847efe5ea7a5,78909434,None,None,1806.90.00,Outros,00000078909434,FERRERO ROCHER,https://cdn-cosmos.bluesoft.com.br/products/78...,78909434,Unidade,1,OK,AUDITADO,BOMBOM FERRERO ROCHER 3X16G,1539995834000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8ee42cd3-4200-45dc-a10f-6b9e62bc5127,7898917945100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GTIN_NOT_FOUND,NaN,NaN,1540442673000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bda1d82f-cfad-482c-a243-65ae7f04a7bd,27702018072396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GTIN_NOT_FOUND,NaN,NaN,1540272188000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,427b4261-d73d-4c13-a085-d9d5bb988c39,7896005279595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GTIN_NOT_FOUND,NaN,NaN,1549576419000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df = infomix.set_index("gtin").join(descricoes.set_index("gtin"), how="left", on="gtin")
df = df.join(cosmos.set_index("gtin").rename(columns={'id': 'id_cosmos', 'response.status': 'response.status_cosmos', 'created_at': 'created_at_cosmos'}), how="left", on="gtin")
df = df.join(gs1.set_index("gtin").rename(columns={'id': 'id_gs1', 'response.status': 'response.status_gs1', 'created_at': 'created_at_gs1'}), how="left", on="gtin")
df = df.reset_index().set_index("cnpj").join(cnpj.set_index("cnpj").rename(columns={'id': 'id_cnpj'}), how="left", on="cnpj")


In [37]:
df['response.status_gs1'].value_counts()

OK                  377480
ERROR                 3373
CONNECTION_ERROR      1227
Name: response.status_gs1, dtype: int64

In [43]:
# validacao consiste em verificar se o status na tabela gs1 está OK
df = df.loc[(df['response.status_gs1'] == 'OK')]
df = df.reset_index().drop_duplicates(subset=['gtin', 'cnpj']).head()
df = df[['gtin', 'cnpj', 'response.nome', 'response.municipio', 'response.uf', 'description', 'category']]
df = df.rename(columns={'response.nome': 'razao_social', 'response.municipio': 'municipio', 'response.uf': 'uf', 'description': 'descricao', 'category': 'categoria'})
df.head()

,gtin,cnpj,razao_social,municipio,uf,descricao,categoria
0,6890101502534,8540603000198,SUPERMERCADO LESTE OESTE LTDA,MACEIO,AL,NaN,massas e molhos
1,7898605250417,71322150004408,SAVEGNAGO-SUPERMERCADOS LTDA,SAO CARLOS,SP,CERVEJA COLORADO CAIUM 300ML,cervejas
19,7896000724465,43235985000732,CATRICALA E CIA LIMITADA,SAO JOSE DO RIO PRETO,SP,MASCARA NIELY GOLD MATIZADORA LOIRO ABSOLUTO 430G,cabelo
25,7896102589122,47508411134004,COMPANHIA BRASILEIRA DE DISTRIBUICAO,FORTALEZA,CE,MILHO VERDE S SODIO QUERO LT 200G,conservas e enlatados
61,7895000474646,47508411135833,COMPANHIA BRASILEIRA DE DISTRIBUICAO,CATANDUVA,SP,QA MARSH BRC BAUL50G,bomboniere
